# Part 1 - Extracting and Saving Data from Yelp API

## Obective

- For this CodeAlong, we will be working with the Yelp API. 
- You will use the the Yelp API to search your home town for a cuisine type of your choice.
- Next class, we will then use Plotly Express to create a map with the Mapbox API to visualize the results.
    
    

## Tools You Will Use
- Part 1:
    - Yelp API:
        - Getting Started: 
            - https://www.yelp.com/developers/documentation/v3/get_started

    - `YelpAPI` python package
        -  "YelpAPI": https://github.com/gfairchild/yelpapi
- Part 2:

    - Plotly Express: https://plotly.com/python/getting-started/
        - With Mapbox API: https://www.mapbox.com/
        - `px.scatter_mapbox` [Documentation](https://plotly.com/python/scattermapbox/): 




### Applying Code From
- Efficient API Calls Lesson Link: https://login.codingdojo.com/m/376/12529/88078

In [2]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

## 1. Registering for Required APIs


- Yelp: https://www.yelp.com/developers/documentation/v3/get_started


> Check the official API documentation to know what arguments we can search for: https://www.yelp.com/developers/documentation/v3/business_search

### Load Credentials and Create Yelp API Object

In [3]:
# Load API Credentials
with open('/Users/purvikansara/.secret/yelp_api.json') as f:
    creds = json.load(f)
creds.keys()

dict_keys(['Client-ID', 'API Key'])

In [3]:
creds['API Key']

'HsJ2_Ur3gUElE7C7c_5oj86feE_g573PiBA44nQ1MTVGuWq4FffTxd82I7Hb38ESWKC3CY-JHtfoA7OSdU32DLJbUNfJjiqeJ0jZDnKqk1TqlJ5dxVMKjEa-WaO6YnYx'

In [4]:
# Instantiate YelpAPI Variable
yelp = YelpAPI(creds['API Key'],timeout_s = 5.0)
yelp

### Define Search Terms and File Paths

In [5]:
# set our API call parameters and filename before the first call
TERM = 'Pizza'
LOCATION = 'Seattle, WA 98122'

In [6]:
## Specify fodler for saving data
FOLDER = "Data/"
os.makedirs(FOLDER,exist_ok =True)

# Specifying JSON_FILE filename (can include a folder)
JSON_FILE = FOLDER+ f"{LOCATION.split(',')[0]}-{TERM}.json"

In [7]:
JSON_FILE

'Data/Seattle-Pizza.json'

### Check if Json File exists and Create it if it doesn't

In [8]:
## Check if JSON_FILE exists

## If it does not exist: 

if os.path.isfile(JSON_FILE) ==False:   
    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder  = os.path.dirname(JSON_FILE)
    
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok = True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print("File not found. Saving empy list to file.")
    
    
    ## save the first page of results
    with open(JSON_FILE,'w') as f:
        json.dump([],f)
        
## If it exists, inform user
else:
    print("{JSON_FILE} already exists.")

{JSON_FILE} already exists.


### Load JSON FIle and account for previous results

In [9]:
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
## set offset based on previous results
n_results = len(previous_results)
print(f' -- {n_results} previous result found.')

 -- 0 previous result found.


### Make the first API call to get the first page of data

- We will use this first result to check:
    - how many total results there are?
    - Where is the actual data we want to save?
    - how many results do we get at a time?


In [11]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results+1)
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [12]:
## How many results total?
total_results = results['total']
total_results

841

- Where is the actual data we want to save?

In [13]:
results['businesses']

[{'id': 'FVbwpNA1uZEGiM02N4XtUg',
  'alias': 'blotto-seattle',
  'name': 'Blotto',
  'image_url': 'https://s3-media3.fl.yelpcdn.com/bphoto/Q4Y1CwKmapk89MYpoxCu7Q/o.jpg',
  'is_closed': False,
  'url': 'https://www.yelp.com/biz/blotto-seattle?adjust_creative=r8Qlusnw5U_OXTmzDH90Ug&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=r8Qlusnw5U_OXTmzDH90Ug',
  'review_count': 24,
  'categories': [{'alias': 'pizza', 'title': 'Pizza'}],
  'rating': 5.0,
  'coordinates': {'latitude': 47.61849, 'longitude': -122.31664},
  'transactions': [],
  'location': {'address1': '1830 12th Ave',
   'address2': '',
   'address3': None,
   'city': 'Seattle',
   'zip_code': '98122',
   'country': 'US',
   'state': 'WA',
   'display_address': ['1830 12th Ave', 'Seattle, WA 98122']},
  'phone': '+12064031809',
  'display_phone': '(206) 403-1809',
  'distance': 1686.8621759079958},
 {'id': 'Wi6LFkjIausYj277ru6pqg',
  'alias': 'humble-pie-seattle',
  'name': 'Humble Pie',
  'image_url': 'http

In [14]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page

20

- Calculate how many pages of results needed to cover the total_results

In [15]:
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

43

In [17]:
for i in tqdm_notebook( range(1,n_pages+1)):
    ## The block of code we want to TRY to run
    try:
        
        time.sleep(.2)
        
        ## Read in results in progress file and check the length
        with open(JSON_FILE, 'r') as f:
            previous_results = json.load(f)
            
        ## save number of results for to use as offset
        n_results = len(previous_results)
        
        
        ## use n_results as the OFFSET 
        results = yelp.search_query(location=LOCATION,
                                        term=TERM, 
                                        offset=n_results+1)

        ## append new results and save to file
        previous_results.extend(results['businesses'])

        with open(JSON_FILE,'w') as f:
            json.dump(previous_results,f)
            
    ## What to do if we get an error/exception.
    except Exception as e: # saving the error message so we can print it.
        print('[!] ERROR: ',e)

  0%|          | 0/43 [00:00<?, ?it/s]

## Open the Final JSON File with Pandas

In [18]:
df = pd.read_json(JSON_FILE)
df

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
0,FVbwpNA1uZEGiM02N4XtUg,blotto-seattle,Blotto,https://s3-media3.fl.yelpcdn.com/bphoto/Q4Y1Cw...,False,https://www.yelp.com/biz/blotto-seattle?adjust...,24,"[{'alias': 'pizza', 'title': 'Pizza'}]",5.0,"{'latitude': 47.61849, 'longitude': -122.31664}",[],"{'address1': '1830 12th Ave', 'address2': '', ...",+12064031809,(206) 403-1809,1686.862176,NaN
1,Wi6LFkjIausYj277ru6pqg,humble-pie-seattle,Humble Pie,https://s3-media1.fl.yelpcdn.com/bphoto/9tK83_...,False,https://www.yelp.com/biz/humble-pie-seattle?ad...,344,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,"{'latitude': 47.5976491915013, 'longitude': -1...",[delivery],"{'address1': '525 Rainier Ave S', 'address2': ...",+12063295133,(206) 329-5133,2166.834011,$$
2,EwrAoy7XbX8nfemZdDvupA,cornelly-seattle,Cornelly,https://s3-media4.fl.yelpcdn.com/bphoto/HlYYBT...,False,https://www.yelp.com/biz/cornelly-seattle?adju...,77,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 47.624577, 'longitude': -122.325577}","[pickup, delivery]","{'address1': '601 Summit Ave E', 'address2': '...",,,2578.701393,NaN
3,ugTsEtjvwRhteac_6JcuPw,italian-family-pizza-seattle,Italian Family Pizza,https://s3-media2.fl.yelpcdn.com/bphoto/SgiSrJ...,False,https://www.yelp.com/biz/italian-family-pizza-...,966,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"{'latitude': 47.60937, 'longitude': -122.32546}","[pickup, delivery]","{'address1': '1028 Madison St', 'address2': No...",+12065380040,(206) 538-0040,2271.307105,$$
4,Ss_66zV-zhK_57_6MgV93w,aglio-e-olio-seattle,Aglio e Olio,https://s3-media2.fl.yelpcdn.com/bphoto/inrSl1...,False,https://www.yelp.com/biz/aglio-e-olio-seattle?...,267,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.5,"{'latitude': 47.62323, 'longitude': -122.29593}",[delivery],"{'address1': '2743 E Madison St', 'address2': ...",+12067098883,(206) 709-8883,1130.900743,$$
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
835,7tQY7TW-oYhAztdpUVgfZQ,subway-seattle-127,Subway,https://s3-media2.fl.yelpcdn.com/bphoto/KFiJVP...,False,https://www.yelp.com/biz/subway-seattle-127?ad...,39,"[{'alias': 'sandwiches', 'title': 'Sandwiches'}]",3.0,"{'latitude': 47.667511119207, 'longitude': -12...","[pickup, delivery]","{'address1': '5300 15th Ave NW', 'address2': '...",+12067895434,(206) 789-5434,8529.998816,$
836,rcDbUcs83gvoRxhaTmFfnQ,cypress-lounge-and-wine-bar-bellevue,Cypress Lounge & Wine Bar,https://s3-media2.fl.yelpcdn.com/bphoto/DsFhIb...,False,https://www.yelp.com/biz/cypress-lounge-and-wi...,63,"[{'alias': 'wine_bars', 'title': 'Wine Bars'},...",3.5,"{'latitude': 47.6157033283809, 'longitude': -1...",[],"{'address1': '600 Bellevue Way NE', 'address2'...",+14256381000,(425) 638-1000,7123.208902,$$
837,G-p5SgQnBM5e8XxMMO1qOg,pho-aroma-seattle-2,Pho Aroma,https://s3-media1.fl.yelpcdn.com/bphoto/BCxDac...,False,https://www.yelp.com/biz/pho-aroma-seattle-2?a...,384,"[{'alias': 'vietnamese', 'title': 'Vietnamese'...",4.0,"{'latitude': 47.55172, 'longitude': -122.36327}",[delivery],"{'address1': '5605 Delridge Way SW', 'address2...",+12069324343,(206) 932-4343,8499.808775,$$
838,8S3qU0TGMJDyj4KHBmxlaw,pho-tai-kirkland,Pho Tai,https://s3-media3.fl.yelpcdn.com/bphoto/tnvGkA...,False,https://www.yelp.com/biz/pho-tai-kirkland?adju...,184,"[{'alias': 'vietnamese', 'title': 'Vietnamese'...",3.5,"{'latitude': 47.676215, 'longitude': -122.205084}","[delivery, pickup]","{'address1': '147 Park Ln', 'address2': '', 'a...",+14255765407,(425) 576-5407,9765.353049,$$


In [19]:
## convert the filename to a .csv.gz
csv_file = JSON_FILE.replace('.json','.csv.gz')
csv_file

'Data/Seattle-Pizza.csv.gz'

In [20]:
## Save it as a compressed csv (to save space)
df.to_csv(csv_file, compression='gzip', index=False)

## Bonus: compare filesize with os module's `os.path.getsize`

In [21]:
size_json = os.path.getsize(JSON_FILE)
size_csv_gz = os.path.getsize(JSON_FILE.replace('.json','.csv.gz'))

print(f'JSON FILE: {size_json:,} Bytes')
print(f'CSV.GZ FILE: {size_csv_gz:,} Bytes')

print(f'the csv.gz is {size_json/size_csv_gz} times smaller!')

JSON FILE: 818,946 Bytes
CSV.GZ FILE: 118,546 Bytes
the csv.gz is 6.908255023366457 times smaller!


## Next Class: Processing the Results and Mapping 